Archivo para probar los prompt

In [16]:
%pip install pandas
import pandas as pd
import json

pacientes_path = '../01_setup/data/json/pacientes.json'
evolucion_path = '../01_setup/data/json/evolucion.json'

#Cargar datos de los pacientes
with open(pacientes_path, 'r', encoding='utf-8') as f:
    pacientes_data = json.load(f)
df_pacientes = pd.DataFrame(pacientes_data[2]['data'])

#Cargas las evoluciones
with open(evolucion_path, 'r', encoding='utf-8') as f:
    evolucion_data = json.load(f)
df_evolucion = pd.DataFrame(evolucion_data[2]['data'])

#Fusion
df_pacientes['id_paciente'] = df_pacientes['id_paciente'].astype(int)
df_evolucion['id_paciente'] = df_evolucion['id_paciente'].astype(int)
df_combinado = pd.merge(df_pacientes, df_evolucion, on='id_paciente', how='left')



Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


Formateo de texto para LLM

In [ ]:
import pandas as pd

# Convertir la columna 'fecha' en la tabla de evolucion a formato de fecha
df_combinado['fecha'] = pd.to_datetime(df_combinado['fecha'], errors='coerce')

# Diccionario para almacenar los prompts de cada paciente
prompts_por_paciente = {}

# Agrupar el DataFrame por id_paciente
pacientes_agrupados = df_combinado.groupby('id_paciente')

# Iterar sobre cada grupo de paciente
for id_paciente, grupo_paciente in pacientes_agrupados:
    # Obtener los datos del paciente
    paciente = grupo_paciente.iloc[0]

    # Ordenar las evoluciones del paciente por fecha y eliminar los valores NaT
    # La eliminación de NaT previene el error 'AttributeError: float'
    evoluciones_ordenadas = grupo_paciente.dropna(subset=['fecha']).sort_values(by='fecha')

    # Construir la sección de evoluciones como una cadena de texto
    evoluciones_texto = ""
    if not evoluciones_ordenadas.empty:
        for index, evolucion in evoluciones_ordenadas.iterrows():
            evoluciones_texto += (
                f"- Fecha: {evolucion['fecha'].strftime('%Y-%m-%d')}\n"
                f"  Motivo: {evolucion['motivo']}\n"
                f"  Conducta: {evolucion['conducta']}\n\n"
            )
    else:
        evoluciones_texto = "No se encontraron evoluciones para este paciente.\n\n"

    # Crear el prompt completo del paciente
    prompt_completo = f"""Historia Clínica del Paciente:
- Nombre: {paciente['nombre']} {paciente['apellido']}
- DNI: {paciente['dni']}
- Fecha de Nacimiento: {paciente['fecha_nac']}
- Sexo: {paciente['sexo']}
- Especialidad: {paciente['especialidad']}
- Diagnóstico Principal: {paciente['diagnostico']}
- Enfermedad de Base: {paciente['enfermedad_base']}
- Antecedentes Perinatales: {paciente['ant_perinatales']}
- Antecedentes Familiares: {paciente['ant_familiares']}

Evoluciones del Paciente:
{evoluciones_texto}
"""
prompts_por_paciente[id_paciente] = prompt_completo

# Ejemplo de cómo se vería un prompt
print("--- Ejemplo de un prompt generado ---")
# Puedes cambiar el ID para ver el prompt de otro paciente
ejemplo_prompt = prompts_por_paciente.get(150, "No se encontró el paciente con el ID 2.")
print(ejemplo_prompt)


--- Ejemplo de un prompt generado ---
Historia Clínica del Paciente:
- Nombre: Tadeo Buscaglia
- DNI: 52.029.548
- Fecha de Nacimiento: 2012-02-06
- Sexo: Varón
- Especialidad: Pediátrico
- Diagnóstico Principal: Otra
- Enfermedad de Base: Klippel Trenaunay
- Antecedentes Perinatales: RNT PAEG vaginal Klippel Trenaunay Dx realizado al momento del nacer con uso de media de compresión
- Antecedentes Familiares: No refiere

Evoluciones del Paciente:
- Fecha: 2025-09-02
  Motivo: Control de salud
  Conducta: 




Se han generado los prompts y se han guardado en 'historiales_clinicos_formateados.txt'
